In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np

# 1. Chargement du fichier
# Assuming the file is in the root of your Google Drive, adjust the path if needed.
df = pd.read_csv('/content/drive/MyDrive/Besoin_Client_3/vessel-total-cleaned.csv', parse_dates=['BaseDateTime'])
print(f" Données initiales : {df.shape[0]} lignes")

# 2. Nettoyage des doublons
df = df.drop_duplicates()

# 3. Suppression des lignes essentielles incomplètes
colonnes_importantes = ['LAT', 'LON', 'BaseDateTime', 'SOG', 'COG', 'Heading']
df = df.dropna(subset=colonnes_importantes)

# 4. Filtrage des valeurs aberrantes
df = df[(df['SOG'] >= 0) & (df['SOG'] <= 50)]
df = df[(df['LAT'] >= -90) & (df['LAT'] <= 90)]
df = df[(df['LON'] >= -180) & (df['LON'] <= 180)]

# 5. Garder uniquement les navires ayant au moins 20 points
mmsi_counts = df['MMSI'].value_counts()
mmsi_valides = mmsi_counts[mmsi_counts > 20].index
df = df[df['MMSI'].isin(mmsi_valides)].reset_index(drop=True)

print(f" Données nettoyées : {df.shape[0]} lignes, {df['MMSI'].nunique()} navires restants")

# 6. Sélection aléatoire de 5 navires
np.random.seed(42)
mmsi_selectionnes = np.random.choice(df['MMSI'].unique(), size=5, replace=False)

# 7. Affichage des noms de navires et création d’un dictionnaire
navires = {}
print("\n Navires sélectionnés :")
for mmsi in mmsi_selectionnes:
    nom = df[df['MMSI'] == mmsi]['VesselName'].dropna().unique()
    nom_affiche = nom[0] if len(nom) > 0 else "Nom inconnu"
    df_navire = df[df['MMSI'] == mmsi].sort_values(by='BaseDateTime').reset_index(drop=True)
    navires[mmsi] = df_navire
    print(f"• MMSI : {mmsi} | Nom : {nom_affiche} ")

# 8. Optionnel : enregistrer la version nettoyée
df.to_csv('vessel-cleaned-final.csv', index=False)

 Données initiales : 229185 lignes
 Données nettoyées : 229185 lignes, 96 navires restants

 Navires sélectionnés :
• MMSI : 636021410 | Nom : STOLT HALCON 
• MMSI : 538009198 | Nom : PACIFIC RUBY 
• MMSI : 368295110 | Nom : HOS ROSEHILL 
• MMSI : 538007356 | Nom : OWL 2 
• MMSI : 538009657 | Nom : CLIPPER MEDWAY 


# Prédiction du  STOLT HALCON

In [11]:
#  1. Importation des bibliothèques
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
import folium

#  2. Chargement des données
df = pd.read_csv('vessel-cleaned-final.csv', parse_dates=['BaseDateTime'])  # Lecture du CSV avec conversion de la date
df_halcon = df[df['VesselName'] == 'STOLT HALCON']                          # Filtrage pour ne garder que STOLT HALCON
df_halcon = df_halcon.sort_values(by='BaseDateTime').reset_index(drop=True) # Tri chronologique et réinitialisation des index

#  3. Encodage des variables catégorielles (texte → nombre)
cat_vars = ['VesselType', 'Status', 'TransceiverClass']       # Colonnes à encoder
for col in cat_vars:
    if col in df_halcon.columns:                              # Vérifie si la colonne existe
        df_halcon[col] = LabelEncoder().fit_transform(df_halcon[col].astype(str))  # Applique l'encodage

#  4. Création de la colonne 'timedelta' (écart de temps en minutes entre lignes)
df_halcon['timedelta'] = df_halcon['BaseDateTime'].diff().dt.total_seconds().div(60).fillna(0)

#  5. Sélection des features (variables explicatives) utilisées pour la prédiction
selected_features = ['SOG', 'COG', 'Heading', 'VesselType', 'Length', 'Draft']

#  6. Paramètres Sliding Window
window_size = 5                      # Nombre de points consécutifs utilisés pour faire une prédiction
forecast_steps = [5, 10, 15]         # Horizons futurs de prédiction (en nombre de points, pas en minutes réelles)
min_timedelta = 3                    # Délai minimal entre deux points consécutifs (en minutes)
max_timedelta = 10                   # Délai maximal accepté

samples = []                         # Liste des X (données d'entrée)
targets = []                         # Liste des Y (valeurs cibles à prédire)
input_windows = []                   # Liste des fenêtres pour affichage des positions d'entrée

# 7. Construction des fenêtres glissantes
for i in range(len(df_halcon) - window_size - max(forecast_steps) + 1):
    window = df_halcon.iloc[i:i+window_size]                # Fenêtre de 5 lignes consécutives
    timedeltas = window['timedelta'].values[1:]             # Liste des écarts de temps (ignore le premier = 0)

    if np.all((timedeltas >= min_timedelta) & (timedeltas <= max_timedelta)):
        x = window[selected_features].values.flatten()      # Aplatit les données de la fenêtre (5 lignes → 1 ligne)
        y = []
        for step in forecast_steps:                         # Pour chaque horizon de prédiction (t+5, t+10...)
            pos_idx = i + window_size + step - 1            # Position future à prédire
            if pos_idx < len(df_halcon):
                y.extend(df_halcon.loc[pos_idx, ['LAT', 'LON']].values)  # Ajoute LAT et LON à prédire
        if len(y) == 2 * len(forecast_steps):
            samples.append(x)                               # Ajoute les features à X
            targets.append(y)                               # Ajoute les cibles à Y
            input_windows.append(df_halcon.iloc[i:i+window_size][['LAT', 'LON', 'SOG', 'COG']])  # Sauvegarde des positions d'entrée

#  8. Conversion des listes en tableaux numpy (X = entrées, Y = cibles)
X = np.array(samples)
Y = np.array(targets)

# 9. Division des données en jeu d’entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 10. Entraînement du modèle Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)                                       # Apprentissage sur les données d'entraînement

#  11. Prédictions sur les données de test
predictions = model.predict(X_test)                               # Prédiction des LAT/LON futurs
mae_total = mean_absolute_error(y_test, predictions)              # Calcul de la MAE globale
r2_total = r2_score(y_test, predictions)                          # Calcul du coefficient de détermination

print(f"\n MAE globale (LAT & LON, tous horizons) : {mae_total:.6f}")
print(f" R² global (coefficient de détermination) : {r2_total:.6f}")

#  12. MAE détaillée par horizon (t+5, t+10, ...)
# (Suppression de l'affichage des MAE par horizon)

# 13. Affichage des données d'entrée et des prédictions d'un exemple
print("\n  5 positions consécutives LAT/LON/SOG/COG d'entrée :")
print(input_windows[-1].to_string(index=False))

print("\n  Positions prédites pour t+5, t+10, t+15 :")
pred_sample = model.predict(X_test[:1])[0]  # prendre un exemple parmi les données test
for j, step in enumerate(forecast_steps):
    lat = pred_sample[2 * j]
    lon = pred_sample[2 * j + 1]
    print(f"   → t+{step} min : LAT = {lat:.6f} / LON = {lon:.6f}")


with open('/content/drive/MyDrive/Besoin_Client_3/model_stolt_halcon.pkl', 'wb') as f:
    pickle.dump(model, f)
print(" Modèle sauvegardé sous : model_stolt_halcon.pkl")


 MAE globale (LAT & LON, tous horizons) : 0.027244
 R² global (coefficient de détermination) : 0.748093

  5 positions consécutives LAT/LON/SOG/COG d'entrée :
     LAT       LON  SOG   COG
29.32428 -94.61328  0.0 312.7
29.32431 -94.61330  0.0 312.7
29.32437 -94.61326  0.1 312.7
29.32438 -94.61332  0.1 312.7
29.32437 -94.61328  0.0 312.7

  Positions prédites pour t+5, t+10, t+15 :
   → t+5 min : LAT = 29.743963 / LON = -95.098943
   → t+10 min : LAT = 29.743973 / LON = -95.098931
   → t+15 min : LAT = 29.743954 / LON = -95.098940
 Modèle sauvegardé sous : model_stolt_halcon.pkl


# Prédiction et comparaison des 5 navires

In [12]:
# 1. Imports : bibliothèques nécessaires
import pandas as pd  # manipulation des données
import numpy as np   # calculs numériques
from sklearn.ensemble import RandomForestRegressor  # modèle de régression
from sklearn.model_selection import train_test_split  # séparation train/test
from sklearn.metrics import mean_absolute_error, r2_score  # métriques de performance
from sklearn.preprocessing import LabelEncoder  # encodage des variables catégorielles
import joblib  # pour sauvegarder les modèles
import os  # gestion de fichiers et dossiers

# 2. Chargement des données
df = pd.read_csv('vessel-cleaned-final.csv', parse_dates=['BaseDateTime'])  # lecture du CSV avec format datetime

# 3. Liste des MMSI sélectionnés (navires spécifiques)
mmsi_list = [636021410, 538009198, 368295110, 538007356, 538009657]

# 4. Paramètres de modélisation
selected_features = ['SOG', 'COG', 'Heading', 'VesselType', 'Length', 'Draft']  # variables d’entrée
cat_vars = ['VesselType', 'Status', 'TransceiverClass']  # variables catégorielles à encoder
window_size = 5  # taille de la fenêtre glissante (5 lignes consécutives)
forecast_steps = [5, 10, 15]  # temps de prédiction en minutes
min_timedelta = 3  # temps min entre 2 points (en minutes)
max_timedelta = 10  # temps max entre 2 points (en minutes)

# 5. Création du dossier pour sauvegarder les modèles
models_dir = '/content/drive/MyDrive/Besoin_Client_3/models'
os.makedirs(models_dir, exist_ok=True)

# 6. Initialisation des résultats
results = []

# 7. Boucle d'entraînement par navire
for mmsi in mmsi_list:
    # Données d’un navire triées par temps
    df_ship = df[df['MMSI'] == mmsi].sort_values('BaseDateTime').reset_index(drop=True)

    # Encodage des variables catégorielles
    encoders = {}
    for col in cat_vars:
        if col in df_ship.columns:
            le = LabelEncoder()
            df_ship[col] = le.fit_transform(df_ship[col].astype(str))
            encoders[col] = le  # on stocke l’encodeur pour ce navire

    # Calcul du temps entre les positions
    df_ship['timedelta'] = df_ship['BaseDateTime'].diff().dt.total_seconds().div(60).fillna(0)

    # Initialisation des données d’entrée/sortie
    samples, targets = [], []
    input_windows = []

    # Sliding window sur les trajectoires
    for i in range(len(df_ship) - window_size - max(forecast_steps) + 1):
        window = df_ship.iloc[i:i + window_size]
        timedeltas = window['timedelta'].values[1:]

        # On accepte uniquement les fenêtres avec des écarts de temps cohérents
        if np.all((timedeltas >= min_timedelta) & (timedeltas <= max_timedelta)):
            x = window[selected_features].values.flatten()  # les features d’entrée
            y = []

            # Construction des cibles à t+5,10,15
            for step in forecast_steps:
                pos_idx = i + window_size + step - 1
                if pos_idx < len(df_ship):
                    y.extend(df_ship.loc[pos_idx, ['LAT', 'LON']].values)

            # Vérifie que toutes les positions futures existent
            if len(y) == 2 * len(forecast_steps):
                samples.append(x)
                target_columns = []
                for step in forecast_steps:
                    target_columns += [f'LAT_t+{step}', f'LON_t+{step}']
                targets.append(pd.Series(y, index=target_columns))
                input_windows.append(df_ship.iloc[i:i + window_size][['LAT', 'LON', 'SOG', 'COG']])

    # Si pas assez de données, on ignore le navire
    if len(samples) < 50:
        print(f" Trop peu d’exemples pour le MMSI {mmsi}, ignoré.")
        continue

    # Conversion des données
    X = np.array(samples)
    Y = pd.DataFrame(targets)

    # Séparation train/test
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # Modèle Random Forest
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)  # entraînement
    predictions = model.predict(X_test)  # prédiction sur test

    # Évaluation
    mae_global = mean_absolute_error(y_test, predictions)
    r2_global = r2_score(y_test, predictions)

    # Nom du navire (si disponible)
    nom = df_ship['VesselName'].dropna().unique()
    nom = nom[0] if len(nom) > 0 else f"Navire_{mmsi}"

    # Affichage des résultats
    print(f"\n Navire : {nom} (MMSI : {mmsi})")
    print(f" - MAE globale : {mae_global:.6f}")
    print(f" - R² global   : {r2_global:.6f}")

    print("\n Exemple d'entrée (5 lignes consécutives) :")
    print(input_windows[-1].to_string(index=False))

    print("\n Exemple de prédiction (t+5, t+10, t+15) :")
    pred_sample = model.predict(X_test[:1])[0]
    for j, step in enumerate(forecast_steps):
        lat = pred_sample[2 * j]
        lon = pred_sample[2 * j + 1]
        print(f" → t+{step} min : LAT = {lat:.6f} / LON = {lon:.6f}")

    # Sauvegarde du modèle et des encodages
    joblib.dump(model, f"{models_dir}/model_mmsi_{mmsi}.pkl")
    joblib.dump(encoders, f"{models_dir}/encoders_mmsi_{mmsi}.pkl")

    # Enregistrement des résultats
    results.append({
        "MMSI": mmsi,
        "Nom": nom,
        "MAE_global": mae_global,
        "R2_global": r2_global
    })

# Résumé des résultats pour tous les navires
results_df = pd.DataFrame(results)
print("\n Comparaison des performances par navire :")
print(results_df)

# Export des résultats en CSV
results_df.to_csv("resultats_selection_navires.csv", index=False)
print("\n Fichier 'resultats_selection_navires.csv' enregistré.")



 Navire : STOLT HALCON (MMSI : 636021410)
 - MAE globale : 0.027244
 - R² global   : 0.748093

 Exemple d'entrée (5 lignes consécutives) :
     LAT       LON  SOG   COG
29.32428 -94.61328  0.0 312.7
29.32431 -94.61330  0.0 312.7
29.32437 -94.61326  0.1 312.7
29.32438 -94.61332  0.1 312.7
29.32437 -94.61328  0.0 312.7

 Exemple de prédiction (t+5, t+10, t+15) :
 → t+5 min : LAT = 29.743963 / LON = -95.098943
 → t+10 min : LAT = 29.743973 / LON = -95.098931
 → t+15 min : LAT = 29.743954 / LON = -95.098940

 Navire : PACIFIC RUBY (MMSI : 538009198)
 - MAE globale : 0.000109
 - R² global   : 0.999998

 Exemple d'entrée (5 lignes consécutives) :
     LAT       LON  SOG   COG
29.72962 -95.02208  0.0 173.7
29.72960 -95.02213  0.0 173.7
29.72956 -95.02214  0.0 173.7
29.72958 -95.02211  0.1 173.7
29.72958 -95.02210  0.0 173.7

 Exemple de prédiction (t+5, t+10, t+15) :
 → t+5 min : LAT = 29.244238 / LON = -94.453610
 → t+10 min : LAT = 29.244276 / LON = -94.453668
 → t+15 min : LAT = 29.244294

In [13]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# === 1. Paramètres ===
file_path = "/content/drive/MyDrive/Besoin_Client_3/vessel-cleaned-final.csv"
models_dir = "/content/drive/MyDrive/Besoin_Client_3/models"
os.makedirs(models_dir, exist_ok=True)

features = ['SOG', 'COG', 'Heading', 'VesselType', 'Length', 'Draft']
window_size = 5
forecast_steps = [5, 10, 15]
min_timedelta = 3
max_timedelta = 10

# === 2. Chargement des données ===
df = pd.read_csv(file_path, parse_dates=["BaseDateTime"])

# === 3. Choix du MMSI ===
available_mmsi = df['MMSI'].dropna().unique().tolist()
print("MMSI disponibles :", available_mmsi[:10], "...")
mmsi_input = input("Entrez un MMSI à prédire : ").strip()
try:
    mmsi = int(mmsi_input)
except:
    print(" MMSI invalide.")
    exit()

df_ship = df[df['MMSI'] == mmsi].sort_values("BaseDateTime").reset_index(drop=True)
if len(df_ship) < window_size + max(forecast_steps):
    print(" Pas assez de données pour ce navire.")
    exit()

# === 4. Encodage des colonnes catégorielles ===
cat_vars = ['VesselType', 'Status', 'TransceiverClass']
encoder_path = os.path.join(models_dir, f"encoders_mmsi_{mmsi}.pkl")
if os.path.exists(encoder_path):
    encoders = joblib.load(encoder_path)
    for col in cat_vars:
        if col in df_ship.columns and col in encoders:
            df_ship[col] = df_ship[col].astype(str)
            df_ship[col] = df_ship[col].apply(lambda x: x if x in encoders[col].classes_ else encoders[col].classes_[0])
            df_ship[col] = encoders[col].transform(df_ship[col])
else:
    print(" Avertissement : aucun encodeur enregistré trouvé. Les colonnes catégorielles ne seront pas encodées correctement.")

# === 5. Création des fenêtres ===
df_ship["timedelta"] = df_ship["BaseDateTime"].diff().dt.total_seconds().div(60).fillna(0)
samples, targets, input_windows = [], [], []
for i in range(len(df_ship) - window_size - max(forecast_steps) + 1):
    window = df_ship.iloc[i:i + window_size]
    timedeltas = window["timedelta"].values[1:]
    if np.all((timedeltas >= min_timedelta) & (timedeltas <= max_timedelta)):
        x = window[features].values.flatten()
        y = []
        for step in forecast_steps:
            idx = i + window_size + step - 1
            if idx < len(df_ship):
                y.extend(df_ship.loc[idx, ['LAT', 'LON']].values)
        if len(y) == 2 * len(forecast_steps):
            samples.append(x)
            targets.append(y)
            input_windows.append(window[features])

if len(samples) < 5:
    print(" Pas assez d'exemples valides pour entraîner et tester un modèle (minimum 5 requis).")
    exit()

X = np.array(samples)
Y = np.array(targets)

# === 6. Chargement ou entraînement du modèle ===
model_path = os.path.join(models_dir, f"model_mmsi_{mmsi}.pkl")
if os.path.exists(model_path):
    model = joblib.load(model_path)
    print(f" Modèle chargé depuis : {model_path}")
else:
    print(" Aucun modèle existant, entraînement en cours...")
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    joblib.dump(model, model_path)
    print(f" Modèle sauvegardé dans : {model_path}")

# === 7. Prédiction et affichage ===
X_input = X[-1].reshape(1, -1)
y_pred = model.predict(X_input)[0]
display_window = input_windows[-1]

print(f"\n Prédiction pour le MMSI : {mmsi}")
print("\n Données utilisées pour la prédiction :")
print(display_window.to_string(index=False))
print("\n Prédictions des positions futures :")
for j, step in enumerate(forecast_steps):
    print(f" → t+{step} min : LAT = {y_pred[2*j]:.6f}, LON = {y_pred[2*j+1]:.6f}")


MMSI disponibles : [636017833, 367477280, 366954420, 367670000, 255805583, 566996000, 319963000, 538007204, 538005585, 244561000] ...
Entrez un MMSI à prédire : 636017833
 Avertissement : aucun encodeur enregistré trouvé. Les colonnes catégorielles ne seront pas encodées correctement.
 Aucun modèle existant, entraînement en cours...
 Modèle sauvegardé dans : /content/drive/MyDrive/Besoin_Client_3/models/model_mmsi_636017833.pkl

 Prédiction pour le MMSI : 636017833

 Données utilisées pour la prédiction :
 SOG  COG  Heading  VesselType  Length  Draft
 0.0  9.8       53          80     183   10.0
 0.0  9.8       53          80     183   10.0
 0.0  9.8       53          80     183   10.0
 0.0  9.8       53          80     183   10.0
 0.0  9.8       53          80     183   10.0

 Prédictions des positions futures :
 → t+5 min : LAT = 29.722854, LON = -95.235862
 → t+10 min : LAT = 29.722861, LON = -95.235858
 → t+15 min : LAT = 29.722864, LON = -95.235863
